# Transformer based models

### Importing the required libraries

In [ ]:
! pip install allennlp
! pip install allennlp-models
! pip install nltk==3.5

In [1]:
import io
import numpy as np
# from google_trans_new import google_translator 
from scipy.spatial.distance import cosine
import json 
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize
from itertools import groupby 
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install indic-nlp-library
import indicnlp
from indicnlp.tokenize import indic_tokenize 
!pip install -U torch
import torch
!pip install -U transformers
!pip install -U sentence-transformers
import transformers
from sentence_transformers import SentenceTransformer,util
import numpy as np
!pip install google_trans_new
from google_trans_new import google_translator

In [ ]:
!python --version

Python 3.7.10


In [ ]:
# !pip install stanfordnlp
import stanfordnlp 
stanfordnlp.download('hi')
nlp = stanfordnlp.Pipeline(processors = "tokenize,pos",lang = 'hi')

In [ ]:
# model = SentenceTransformer('stsb-xlm-r-multilingual')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
model = SentenceTransformer('LaBSE')

In [ ]:
from allennlp.predictors import Predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
downloading: 100%|##########| 711853497/711853497 [00:14<00:00, 50727898.53B/s]
/usr/local/lib/python3.7/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:60: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning,
downloading: 100%|##########| 374434792/374434792 [00:09<00:00, 38879752.03B/s]


In [ ]:
#@title
results = predictor.predict(sentence="position held Member of the 11th Lok Sabha, Rajya Sabha")
named_entities = []
flag = 0
named = ""
for word, tag in zip(results["words"], results["tags"]):
    if tag != 'O':
      flag = 1
      named = named+ word + " "
    else:
      if flag:
        named_entities.append(named.strip(' '))
      named = ""
      flag = 0
    print(f"{word}\t{tag}")
if flag:
  named_entities.append(named.strip(' '))
  named = ""
named_entities

position	O
held	O
Member	O
of	O
the	O
11th	O
Lok	B-PER
Sabha	L-PER
,	O
Rajya	B-PER
Sabha	L-PER


['Lok Sabha', 'Rajya Sabha']

In [ ]:
pos_dict = {
'CC': 'coordinating conjunction','CD': 'cardinal digit','DT': 'determiner',
'EX': 'existential there (like: \"there is\" ... think of it like \"there exists\")',
'FW': 'foreign word','IN':  'preposition/subordinating conjunction','JJ': 'adjective \'big\'',
'JJR': 'adjective, comparative \'bigger\'','JJS': 'adjective, superlative \'biggest\'',
'LS': 'list marker 1)','MD': 'modal could, will','NN': 'noun, singular \'desk\'',
'NNS': 'noun plural \'desks\'','NNP': 'proper noun, singular \'Harrison\'',
'NNPS': 'proper noun, plural \'Americans\'','PDT': 'predeterminer \'all the kids\'',
'POS': 'possessive ending parent\'s','PRP': 'personal pronoun I, he, she',
'PRP$': 'possessive pronoun my, his, hers','RB': 'adverb very, silently,',
'RBR': 'adverb, comparative better','RBS': 'adverb, superlative best',
'RP': 'particle give up','TO': 'to go \'to\' the store.','UH': 'interjection errrrrrrrm',
'VB': 'verb, base form take','VBD': 'verb, past tense took',
'VBG': 'verb, gerund/present participle taking','VBN': 'verb, past participle taken',
'VBP': 'verb, sing. present, non-3d take','VBZ': 'verb, 3rd person sing. present takes',
'WDT': 'wh-determiner which','WP': 'wh-pronoun who, what','WP$': 'possessive wh-pronoun whose',
'WRB': 'wh-abverb where, when','QF' : 'quantifier, bahut, thoda, kam (Hindi)','VM' : 'main verb',
'PSP' : 'postposition, common in indian langs','DEM' : 'demonstrative, common in indian langs'
}

#extract parts of speech
def extract_pos(doc):
    pos = [(wrd.text,wrd.pos) for sent in doc.sentences for wrd in sent.words]
    return pos

In [ ]:
# 'सौमिली बिस्वास एक भारतीय फिल्म और टेलीविज़न अभिनेत्री, मॉडल और टीवी प्रस्तोता हैं': [('Soumili Biswas',
#    'occupation',
#    'model'),
#   ('Soumili Biswas', 'occupation', 'actor'),
#   ('Soumili Biswas', 'country of citizenship', 'India'),
#   ('Soumili Biswas', 'occupation', 'television actor')],

In [ ]:
hindi = nlp('ए राजा एक भारतीय राजनीतिज्ञ तथा वे भारत सरकार की पंद्रहवीं लोकसभा के मंत्रीमंडल')
extract_pos(hindi)

[('ए', 'NNPC'),
 ('राजा', 'NNP'),
 ('एक', 'QC'),
 ('भारतीय', 'JJ'),
 ('राजनीतिज्ञ', 'NN'),
 ('तथा', 'CC'),
 ('वे', 'PRP'),
 ('भारत', 'NNPC'),
 ('सरकार', 'NNP'),
 ('की', 'PSP'),
 ('पंद्रहवीं', 'NNP'),
 ('लोकसभा', 'NN'),
 ('के', 'PSP'),
 ('मंत्रीमंडल', 'NN')]

In [ ]:
from nltk import RegexpParser
pos_tags = extract_pos(hindi)
patterns = "NP: {<JJ>*<NN|NNS|NNC|NNCS|NNP|NNPS|NNPC|NNPCS>+}"
PChunker = RegexpParser(patterns)
pos_tags = extract_pos(hindi)
output = PChunker.parse(pos_tags)
for subtree in output.subtrees(filter=lambda t: t.label() == 'NP'):
    print(' '.join([x[0] for x in subtree]))
        

ए राजा
भारतीय राजनीतिज्ञ
भारत सरकार
पंद्रहवीं लोकसभा
मंत्रीमंडल


In [ ]:
sentence1 = "A.Raja"
sentence2 =  'ए राजा'
# sentence2 = "एक भारतीय राजनीतिज्ञ था"
words = [w for w in indic_tokenize.trivial_tokenize(sentence2, lang = 'hi') if w not in STOP_WORDS_HI]
print(words)
sentence2 = ' '.join(words)
# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

# {('Surjit Singh Barnala',
#    'country of citizenship',
#    'British India'),
#   ('Surjit Singh Barnala', 'country of citizenship', 'Dominion of India'),
#   ('Surjit Singh Barnala', 'country of citizenship', 'India'),
#   ('Surjit Singh Barnala', 'date of birth', '1925-10-21T00:00:00Z'),
#   ('Surjit Singh Barnala', 'date of death', '2017-01-14T00:00:00Z'),
#   ('Surjit Singh Barnala', 'occupation', 'politician')},

#  'सुरजीत सिंह बरनाला (21 अक्टूबर 1925 – 14 जनवरी 2017) एक भारतीय राजनीतिज्ञ था': {('Surjit Singh Barnala',
#    'occupation',
#    'politician'),
#   ('Surjit Singh Barnala', 'position held', 'Chief Minister of Punjab'),
#   ('Surjit Singh Barnala', 'position held', 'Member of the 11th Lok Sabha'),
#   ('Surjit Singh Barnala', 'position held', 'Member of the 12th Lok Sabha'),
#   ('Surjit Singh Barnala',
#    'position held',
#    'Member of the Punjab Legislative Assembly'),
#   ('Surjit Singh Barnala', 'position held', 'governor of Andhra Pradesh'),
#   ('Surjit Singh Barnala', 'position held', 'governor of Uttarakhand')},

['ए', 'राजा']
Sentence 1: A.Raja
Sentence 2: ए राजा
Similarity score: 0.8068161010742188


In [ ]:
for word in words:
  if word.isdigit():
    print(word)
  else:
    if len(word) > 2:
      print(word , get_nn(word, src_embeddings, src_id2word, tgt_embeddings, tgt_id2word, K=5))

NameError: ignored

### Loading data from json file

In [ ]:
import json 
# Opening JSON file 
testA,testC,testP = open('/content/drive/MyDrive/test_data/testA.json',encoding = 'utf-8') ,open('/content/drive/MyDrive/test_data/testC2.json',encoding = 'utf-8'),open('/content/drive/MyDrive/test_data/testP.json',encoding = 'utf-8')
# testA,testC,testP = open('/content/drive/MyDrive/test_data/testA.json',encoding = 'utf-8') ,open('/content/drive/MyDrive/test_data/testC2.json',encoding = 'utf-8'),open('/content/drive/MyDrive/test_data/politicians.json',encoding = 'utf-8')
dataA,dataC,dataP = json.load(testA),json.load(testC),json.load(testP),
# returns JSON object as  
# a dictionary 
# data = json.load(f) 
# d = {}
# d = data
  
# Closing file 
# f.close()

In [ ]:
for qid in dataC:
  t =  dataC[qid]
  for key in t:
    if key == 'triples':
      triplelist = t[key]
      for l in triplelist:
        if len(l) == 1:
          triplelist.remove(l)

In [ ]:
### Getting a_c_p for getting all the triples
# Opening JSON file 
f = open('/content/drive/MyDrive/a_c_p.json',) 
data = json.load(f) 
acp = {}
acp = data
f.close() 

# Word Overlap functions

In [ ]:

def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [ ]:
dir = '/content/drive/My Drive/Algo Name detection implementation/Text/'
src_path = '/content/drive/My Drive/wiki.hi.align.vec'
tgt_path = '/content/drive/My Drive/wiki.en.align.vec'
nmax = 50000  # maximum number of word embeddings to load

src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

Get nearest Neighbours

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    # print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    targetwordlist = []                               # List of target words for the source word 
    if word in word2id:                               #Check if word is in vocab
      word_emb = src_emb[word2id[word]]
      scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
      k_best = scores.argsort()[-K:][::-1]
      for i, idx in enumerate(k_best):
          # print((scores[idx], tgt_id2word[idx]))      #To give both distance and word
          if tgt_id2word[idx] not in stop_words:
            targetwordlist.append(tgt_id2word[idx].lower())
      return targetwordlist
    else:
      translator = google_translator()
      translate_text = translator.translate(word,lang_tgt='en') 
      transw = translate_text
      if transw not in stop_words:
      # print(word," - Translated - ",transw)
        return [transw.lower()]
      

# Sentence to Triple Matching

### Obtaining sentence and triple embeddings
We obtain the sentence embeddings by taking each sentence, obtaining word embeddings for each word in the sentence, and averaging the word embeddings. For triple embeddings , we average the embeddings for each word in the triple.

In [ ]:
# Matching Triples with sentences
actors_test, cricketers_test, politicians_test = dataA, dataC, dataP
actors, cricketers, politicians = actors_test, cricketers_test, politicians_test

# Creating the dictionary for the test data where key = sentence and value = list of matching triples
actors_test_dict, cricketers_test_dict, politicians_test_dict = {}, {}, {}
for l in actors_test:
    for k, v in actors_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    actors_test_dict[sentence] = set(t)

for l in cricketers_test:
    for k, v in cricketers_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    cricketers_test_dict[sentence] = set(t)

for l in politicians_test:
    for k, v in politicians_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    politicians_test_dict[sentence] = set(t)
# for l in politicians_test:
#   entries = politicians_test[l]
#   subject = entries['personLabel']
#   triple_list = entries['triples']
  
    
len(actors_test_dict),len(cricketers_test_dict),len(politicians_test_dict)

(50, 100, 100)

Cleaning the annotated triples --  We need to prune out certain triples in politicians -- like date of birth, data of death in predicate, British India and Dominion of India in objects.

In [ ]:
temp = {}
for sentence in politicians_test_dict:
  triplelist = politicians_test_dict[sentence]
  new_triple_list = set()
  for triple in triplelist:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      continue
    else:
      new_triple_list.add(triple)
    temp[sentence] = new_triple_list
politicians_test_dict = temp

temp = {}
for sentence in actors_test_dict:
  triplelist = actors_test_dict[sentence]
  new_triple_list = set()
  for triple in triplelist:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      continue
    else:
      new_triple_list.add(triple)
    temp[sentence] = new_triple_list
actors_test_dict = temp

temp = {}
for sentence in cricketers_test_dict:
  triplelist = cricketers_test_dict[sentence]
  new_triple_list = set()
  for triple in triplelist:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      continue
    else:
      new_triple_list.add(triple)
    temp[sentence] = new_triple_list
cricketers_test_dict = temp

Loading ACP.json for training

In [ ]:
f = open('/content/drive/MyDrive/test_data/politicians.json',encoding = 'utf-8')
acp_p = json.load(f)
f = open('/content/drive/MyDrive/test_data/actors.json',encoding = 'utf-8')
acp_a = json.load(f)
f = open('/content/drive/MyDrive/test_data/cricketers.json',encoding = 'utf-8')
acp_c = json.load(f)

In [ ]:
acp_p

{'Q1234662': {'Hindi sitelink': 'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A4%B0%E0%A5%8D%E0%A4%A3_%E0%A4%B8%E0%A4%BF%E0%A4%82%E0%A4%B9',
  'personLabel': 'Karan Singh',
  'sentences': ['कर्ण सिंह (जन्म 1931) भारतीय राजनेता, लेखक और कूटनीतिज्ञ हैं',
   'जम्मू और कश्मीर के महाराजा हरि सिंह और महारानी तारा देवी के प्रत्यक्ष उत्तराधिकारी (युवराज) के रूप में जन्मे डॉ॰ कर्ण सिंह ने अठारह वर्ष की ही उम्र में राजनीतिक जीवन में प्रवेश कर लिया था और वर्ष १९४९ में प्रधानमन्त्री पं॰ जवाहरलाल नेहरू के हस्तक्षेप पर उनके पिता ने उन्हें राजप्रतिनिधि (रीजेंट) नियुक्त कर दिया',
   'इसके पश्चात अगले अठारह वर्षों के दौरान वे राजप्रतिनिधि, निर्वाचित सदर-ए-रियासत और अन्तत: राज्यपाल के पदों पर रहे',
   '१९६७ में डॉ॰ कर्ण सिंह प्रधानमंत्री इंदिरा गाँधी के नेतृत्व में केंद्रीय मंत्रिमंडल में शामिल किए गए',
   'इसके तुरन्त बाद वे भारतीय राष्ट्रीय कांग्रेस के प्रत्याशी के रूप में जम्मू और कश्मीर के उधमपुर संसदीय क्षेत्र से भारी बहुमत से लोक सभा के सदस्य निर्वाचित हुए',
   'इसी क्षेत्र से वे वर्ष १९७१, १९७७ और 

In [ ]:
# Run this cell to randomly sample, else comment it out to run evaluate. This is for creation of train alignment.
from random import sample
keys = sample(list(acp_p.keys()), 100)
acp_p_random_sampled = {}
for key in keys:
  acp_p_random_sampled[key] = acp_p[key]

keys = sample(list(acp_a.keys()), 100)
acp_a_random_sampled = {}
for key in keys:
  acp_a_random_sampled[key] = acp_a[key]

keys = sample(list(acp_c.keys()), 100)
acp_c_random_sampled = {}
for key in keys:
  acp_c_random_sampled[key] = acp_c[key]
acp_p, acp_a, acp_c = acp_p_random_sampled, acp_a_random_sampled, acp_c_random_sampled

politicians_sent= [acp_p[key]['sentences']  for key in acp_p]

# politicians_sent = []
# for key in acp_p:
#   print(acp_p[key]['sentences'])
#   # entries = acp_p[key]
#   # print(entries)
#   politicians_sent.append(acp_p[key]['sentences'])
politicians_trip = [acp_p[key]['triples']  for key in acp_p]


In [ ]:
#@title
# We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
actors_trip, cricketers_trip, politician_trip = [], [], []
actors_sent, cricketers_sent, politician_sent = [], [], []

# Putting actors,cricketers and politicians from a_c_p
# act, cric, pol = acp['a'], acp['c'], acp['p']      # Original acp.json

# Now, we are using better, cleaned , pruned version of acp json, ( cleaned by Shivprasad)
act = acp_a
cric = acp_c
pol = acp_p
############### Actors #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in actors_test:
    eid = actors_test[e]['entity_id']
    for ele in act:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = act[ele]['triples']
            subject = act[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            actors_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in actors_test:
        if eid == actors_test[e]['entity_id']:
            sentence = actors_test[e]['sentence']
            sentence_list.append(sentence)
    actors_sent.append(sentence_list)


############### Cricketers #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in cricketers_test:
    eid = cricketers_test[e]['entity_id']
    for ele in cric:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = cric[ele]['triples']
            subject = cric[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            cricketers_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in cricketers_test:
        if eid == cricketers_test[e]['entity_id']:
            sentence = cricketers_test[e]['sentence']
            sentence_list.append(sentence)
    cricketers_sent.append(sentence_list)


############### Politicians #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in politicians_test:
    eid = politicians_test[e]['entity_id']
    for ele in pol:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = pol[ele]['triples']
            subject = pol[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            politician_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in politicians_test:
        if eid == politicians_test[e]['entity_id']:
            sentence = politicians_test[e]['sentence']
            sentence_list.append(sentence)
    politician_sent.append(sentence_list)

In [ ]:
# We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
actors_trip, cricketers_trip, politician_trip = [], [], []
actors_sent, cricketers_sent, politician_sent = [], [], []

# Putting actors,cricketers and politicians from a_c_p
# act, cric, pol = acp['a'], acp['c'], acp['p']      # Original acp.json

# Now, we are using better, cleaned , pruned version of acp json, ( cleaned by Shivprasad)
act = acp_a
cric = acp_c
pol = acp_p
############### Actors #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in actors_test:
    eid = actors_test[e]['entity_id']
    for ele in act:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = act[ele]['triples']
            subject = act[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip[1]
                obj = trip[2]
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            actors_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in actors_test:
        if eid == actors_test[e]['entity_id']:
            sentence = actors_test[e]['sentence']
            sentence_list.append(sentence)
    actors_sent.append(sentence_list)


############### Cricketers #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in cricketers_test:
    eid = cricketers_test[e]['entity_id']
    for ele in cric:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = cric[ele]['triples']
            subject = cric[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            cricketers_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in cricketers_test:
        if eid == cricketers_test[e]['entity_id']:
            sentence = cricketers_test[e]['sentence']
            sentence_list.append(sentence)
    cricketers_sent.append(sentence_list)


############### Politicians #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in politicians_test:
    eid = politicians_test[e]['entity_id']
    for ele in pol:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = pol[ele]['triples']
            subject = pol[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            politician_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in politicians_test:
        if eid == politicians_test[e]['entity_id']:
            sentence = politicians_test[e]['sentence']
            sentence_list.append(sentence)
    politician_sent.append(sentence_list)

In [ ]:
actors_trip

In [ ]:
pol

In [ ]:
len(actors_sent), len(cricketers_sent),len(politician_sent)

(50, 100, 99)

In [ ]:
len(actors_trip), len(cricketers_trip),len(politician_trip)

(50, 100, 99)

In [ ]:
actors_trip[0]

[('Soumili Biswas', 'place of birth', 'Kolkata'),
 ('Soumili Biswas', 'sex or gender', 'female'),
 ('Soumili Biswas', 'country of citizenship', 'India'),
 ('Soumili Biswas', 'instance of', 'human'),
 ('Soumili Biswas', 'educated at', 'University of Calcutta'),
 ('Soumili Biswas', 'educated at', 'Jogamaya Devi College'),
 ('Soumili Biswas', 'occupation', 'actor'),
 ('Soumili Biswas', 'occupation', 'model'),
 ('Soumili Biswas', 'occupation', 'television actor'),
 ('Soumili Biswas', 'IMDb ID', 'nm3360682'),
 ('Soumili Biswas', 'Freebase ID', '/m/0k9hns6'),
 ('Soumili Biswas', 'languages spoken, written or signed', 'Bengali')]

In [ ]:
#@title
# def matches(sentences,triples):       # Input is a sentence list and triple list for one article
#     matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
#     for sent in sentences:            # For each sent in sentences
#         ent_matchlist = []
#         # words = [w for w in indic_tokenize.trivial_tokenize(sent, lang = 'hi') if w not in STOP_WORDS_HI]
#         # sent_no_stop = ' '.join(words)
#         sent_embed = model.encode(sent, convert_to_tensor=True)
#         for ent in triples:
#             pred = ent[1]
#             obj = ent[2]
#             # print(pred)
#             ent_embed = model.encode(pred + " " + obj, convert_to_tensor=True)
#             cosine_score = util.pytorch_cos_sim(sent_embed, ent_embed)
#             similarity = cosine_score.item()
            
#             if similarity > 0.7:
#               # For evaluation of precision and recall, keep the below 3 lines commented out. They are to append score to matching triples
#                 # ent = list(ent)
#                 # ent.append(similarity)
#                 # ent = tuple(ent)
#                 ent_matchlist.append(ent)
#         if len(ent_matchlist)>0:    
#             matches_dict[sent] = set(ent_matchlist)
#     return matches_dict

# def matches(sentences,triples):       # Input is a sentence list and triple list for one article
#     matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
#     for sent in sentences:            # For each sent in sentences
#         ent_matchlist = []
#         words = [w for w in indic_tokenize.trivial_tokenize(sent, lang = 'hi') if w not in STOP_WORDS_HI]
#         sent_no_stop = ' '.join(words)
#         sent_embed = model.encode(sent_no_stop, convert_to_tensor=True)
#         for ent in triples:
#             subj = ent[0]
#             pred = ent[1]
#             obj = ent[2]
#             # print(pred)
#             ent_embed_1 = model.encode(subj + " " + pred + " " + obj, convert_to_tensor=True)
#             ent_embed_2 = model.encode(pred + " " + obj, convert_to_tensor=True)
#             cosine_score_1 = util.pytorch_cos_sim(sent_embed, ent_embed_1)
#             cosine_score_2 = util.pytorch_cos_sim(sent_embed, ent_embed_2)
#             similarity_1 = cosine_score_1.item()
#             similarity_2 = cosine_score_2.item()
#             similarity = max(similarity_1, similarity_2)
#             if similarity > 0.45:
#                 print(subj + " " + pred + " " + obj + " == ", similarity_1 , ", ", similarity_2)
#               # For evaluation of precision and recall, keep the below 3 lines commented out. They are to append score to matching triples
#                 # ent = list(ent)
#                 # ent.append(similarity)
#                 # ent = tuple(ent)
#                 ent_matchlist.append(ent)
#         if len(ent_matchlist)>0:    
#             matches_dict[sent] = set(ent_matchlist)
#     return matches_dict

In [ ]:
#@title
def matches(sentences,triples):       # Input is a sentence list and triple list for one article
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:            # For each sent in sentences
        ent_matchlist = []
        hindiwords = [w for w in indic_tokenize.trivial_tokenize(sent, lang = 'hi') if w not in STOP_WORDS_HI]
        sent_no_stop = ' '.join(hindiwords)  # Making the snetence without stop words
        sent_embed = model.encode(sent_no_stop, convert_to_tensor=True)
        bucket = []
        # Creating word overlap bucket
        for word in hindiwords:
                if word.isdigit():
                  continue
                else:
                  if len(word) > 2:
                    wordlist = get_nn(word, src_embeddings, src_id2word, tgt_embeddings, tgt_id2word, K=5)
                    bucket.extend(wordlist)

        for ent in triples:
            subj = ent[0]
            pred = ent[1]
            obj = ent[2]
            # print(pred)

            # Finding named entities from triples
            results = predictor.predict(sentence=pred + " " + obj)
            named_entities = [word for word, tag in zip(results["words"], results["tags"]) if tag!='O']
            # named_entities = []
            # prev = ""
            # for word, tag in zip(results["words"], results["tags"]):
            #   if tag !='O':
            #     prev = prev + word + " "
            #   else:
            #     if prev !='':
            #       named_entities.append(prev.strip(' '))
            #     prev = ""
            # if prev!='':
            #   named_entities.append(prev.strip(' '))  
            flag = 0
            if named_entities:
              # If named entities exist in the triple
              named_entities = set([ele.lower() for ele in named_entities])

              if len(set(bucket).intersection(named_entities)) / len(named_entities) >= 0.75:
                flag = 1
                # print(sent,named_entities)
            if named_entities and flag == 0:
              continue
            # ent_embed_1 = model.encode(subj + " " + pred + " " + obj, convert_to_tensor=True)
            ent_embed_2 = model.encode("He/She is "+ pred + " " + obj, convert_to_tensor=True)
            # cosine_score_1 = util.pytorch_cos_sim(sent_embed, ent_embed_1)
            cosine_score_2 = util.pytorch_cos_sim(sent_embed, ent_embed_2)
            # similarity_1 = cosine_score_1.item()
            similarity_2 = cosine_score_2.item()
            # similarity = max(similarity_1, similarity_2)
            similarity = similarity_2
            if similarity > 0.25:
                # print(subj + " " + pred + " " + obj + " == ", similarity_1 , ", ", similarity_2)
              # For evaluation of precision and recall, keep the below 3 lines commented out. They are to append score to matching triples
                # ent = list(ent)
                # ent.append(similarity)
                # ent = tuple(ent)
                ent_matchlist.append(ent)
        if len(ent_matchlist)>0:    
            matches_dict[sent] = set(ent_matchlist)
    return matches_dict

In [ ]:
#@title
def matches_kp(sentences,triples):       # Matches based on keyphrase
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:            # For each sent in sentences
        ent_matchlist = []
        doc = nlp(sent)
        pos_tags = extract_pos(doc)
        patterns = "NP: {<JJ>*<NN|NNS|NNC|NNCS|NNP|NNPS|NNPC|NNPCS>+}"
        PChunker = RegexpParser(patterns)
        output = PChunker.parse(pos_tags)
        key_phrases = []
        for subtree in output.subtrees(filter=lambda t: t.label() == 'NP'):
            key_phrases.append(' '.join([x[0] for x in subtree]))

        for keyphrase in key_phrases:
          for ent in triples:
              pred = ent[1] 
              obj = ent[2]
              keyp_embed = model.encode(keyphrase, convert_to_tensor=True)
              ent_embed = model.encode(pred+" "+obj, convert_to_tensor=True)
              cosine_score = util.pytorch_cos_sim(keyp_embed, ent_embed)
              similarity = cosine_score.item()
              if similarity > 0.35:
                  # print(subj + " " + pred + " " + obj + " == ", similarity_1 , ", ", similarity_2)
                # For evaluation of precision and recall, keep the below 3 lines commented out. They are to append score to matching triples
                  # ent = list(ent)
                  # ent.append(similarity)
                  # ent = tuple(ent)
                  ent_matchlist.append(ent)
        if len(ent_matchlist)>0:    
            matches_dict[sent] = set(ent_matchlist)
    return matches_dict

In [ ]:
# matches_pol = {}
# for sent_list, triple_list in tqdm(zip(politician_sent, politician_trip)):
#     for k, v in matches(sent_list, triple_list).items():
#         matches_pol[k] = v


# matches_act = {}
# for sent_list, triple_list in tqdm(zip(actors_sent, actors_trip)):
#     for k, v in matches(sent_list, triple_list).items():
#         matches_act[k] = v

matches_cric = {}
for sent_list, triple_list in tqdm(zip(cricketers_sent, cricketers_trip)):
    for k, v in matches(sent_list, triple_list).items():
        matches_cric[k] = v

matches_pol = {}
for sent_list, triple_list in tqdm(zip(politician_sent, politician_trip)):
    for k, v in matches(sent_list, triple_list).items():
        matches_pol[k] = v

100it [07:08,  4.29s/it]
99it [08:41,  5.27s/it]


In [ ]:
matches_act

{'अंजलि मुखी भारतीय हिन्दी टेलीविजन अभिनेत्री हैं': {('Anjali Mukhi',
   'country of citizenship',
   'India'),
  ('Anjali Mukhi', 'given name', 'Anjali'),
  ('Anjali Mukhi', 'residence', 'Mumbai')},
 'अरिज फातिमाArij Fatyma एक पाकिस्तानी टीवी अभिनेत्री और मॉडल है': {('Arij Fatyma',
   'country of citizenship',
   'Pakistan'),
  ('Arij Fatyma', 'occupation', 'model'),
  ('Arij Fatyma', 'occupation', 'television producer')},
 'उपेन पटेल (जन्म 16 अगस्त 1981) एक ब्रिटिश भारतीय मॉडल और बॉलीवुड फिल्म अभिनेता हैं': {('Upen Patel',
   'country of citizenship',
   'India'),
  ('Upen Patel', 'occupation', 'model')},
 'उषा शर्मा एक भारतीय अभिनेत्री और नर्तकी है': {('Usha Sharma',
   'country of citizenship',
   'India'),
  ('Usha Sharma', 'family name', 'Sharma'),
  ('Usha Sharma', 'occupation', 'actor'),
  ('Usha Sharma', 'occupation', 'choreographer'),
  ('Usha Sharma', 'occupation', 'dancer'),
  ('Usha Sharma', 'sex or gender', 'female')},
 'कृष्ण भारद्वाज एक भारतीय टीवी कलाकार हैं': {('Krish

### Evaluation : Precision and Recall

In [ ]:
def evaluate(test_dict, matches_dict):
    sum_prec = 0
    for key, val in matches_dict.items():
        tp, fp = 0, 0
        for k, v in test_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        if (tp+fp) != 0:
            prec = tp/(tp + fp)
        else:
            prec = 0
        sum_prec = prec + sum_prec

    sum_rec = 0
    for k, v in test_dict.items():
        rec = 0
        tp, fp = 0, 0
        for key, val in matches_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        rec = tp/len(v)
        sum_rec = rec + sum_rec

    avg_rec, avg_prec = sum_rec/len(test_dict), sum_prec/len(matches_dict)
    return avg_rec, avg_prec


avg_rec_act, avg_prec_act = evaluate(actors_test_dict, matches_act)
avg_rec_cric, avg_prec_cric = evaluate(cricketers_test_dict, matches_cric)
avg_rec_pol, avg_prec_pol = evaluate(politicians_test_dict, matches_pol)

In [ ]:
(avg_rec_act,avg_prec_act)

(0.7666666666666666, 0.6627659574468086)

In [ ]:
(avg_rec_act,avg_prec_act), (avg_rec_cric, avg_prec_cric) , (avg_rec_pol, avg_prec_pol)

((0.7666666666666666, 0.6627659574468086),
 (0.8181818181818182, 0.6869460978835978),
 (0.6294612794612794, 0.5642350557244172))

In [ ]:
AverageRecall = (avg_rec_act + avg_rec_cric + avg_rec_pol)/3
AveragePrecision = (avg_prec_act + avg_prec_cric + avg_prec_pol)/3

AverageRecall, AveragePrecision

(0.6148709315375983, 0.6734387517666726)

In [ ]:
politicians_test_dict

In [ ]:
matches_cric

{'अजीत चंदिला (जन्म ०५ दिसंबर १९८३) हरियाणा, भारत से एक ऑफ स्पिन गेंदबाज है': {('Ajit Chandila',
   'member of sports team',
   'Haryana cricket team'),
  ('Ajit Chandila', 'occupation', 'cricketer'),
  ('Ajit Chandila', 'sport', 'cricket')},
 'अतुल सतीश वासन (जन्म 23 मार्च 1967) एक पूर्व भारतीय क्रिकेट खिलाड़ी है': {('Atul Wassan',
   'occupation',
   'cricketer'),
  ('Atul Wassan', 'sport', 'cricket')},
 'अनुरा टेनेकून (जन्म 29 अक्टूबर 1946) श्रीलंका की पूर्व क्रिकेटर और श्रीलंका की राष्ट्रीय क्रिकेट टीम की वनडे कप्तान हैं': {('Anura Tennekoon',
   'country for sport',
   'Sri Lanka'),
  ('Anura Tennekoon', 'country of citizenship', 'Sri Lanka'),
  ('Anura Tennekoon',
   'member of sports team',
   'Sri Lanka national cricket team'),
  ('Anura Tennekoon', 'occupation', 'cricketer'),
  ('Anura Tennekoon', 'sport', 'cricket')},
 'अबू जायद चौधरी (जन्म ०२ अगस्त १९९३) एक बांग्लादेशी क्रिकेट खिलाड़ी है जो साइल्हेट डिवीजन के लिए खेलते है': {('Abu Jayed',
   'country of citizenship',
   'Ban

In [ ]:
actors_test_dict

{'अंजलि मुखी भारतीय हिन्दी टेलीविजन अभिनेत्री हैं': [('Anjali Mukhi',
   'country of citizenship',
   'India'),
  ('Anjali Mukhi', 'occupation', 'television actor'),
  ('Anjali Mukhi', 'occupation', 'actor')],
 'अरिज फातिमाArij Fatyma एक पाकिस्तानी टीवी अभिनेत्री और मॉडल है': [('Arij Fatyma',
   'occupation',
   'model'),
  ('Arij Fatyma', 'country of citizenship', 'Pakistan'),
  ('Arij Fatyma', 'occupation', 'actor')],
 'उपेन पटेल (जन्म 16 अगस्त 1981) एक ब्रिटिश भारतीय मॉडल और बॉलीवुड फिल्म अभिनेता हैं': [('Upen Patel',
   'occupation',
   'model'),
  ('Upen Patel', 'occupation', 'actor'),
  ('Upen Patel', 'country of citizenship', 'India')],
 'उषा शर्मा एक भारतीय अभिनेत्री और नर्तकी है': [('Usha Sharma',
   'occupation',
   'actor'),
  ('Usha Sharma', 'country of citizenship', 'India'),
  ('Usha Sharma', 'occupation', 'dancer')],
 'कृष्ण भारद्वाज एक भारतीय टीवी कलाकार हैं': [('Krishna Bharadwaj',
   'occupation',
   'actor'),
  ('Krishna Bharadwaj', 'country of citizenship', 'India')

### Output

In [ ]:
len(matches_act),len(matches_cric),len(matches_pol)

So, 7 sentences didn't find any matches at all.Let's see which ones

In [ ]:
for sent in politicians_test_dict:
  if sent not in matches_pol:
    print(sent," : ", politicians_test_dict[sent] )

Saving as json

In [ ]:
# for sent in actors_test_dict:
#   actors_test_dict[sent] = list(actors_test_dict[sent])
# for sent in cricketers_test_dict:
#   cricketers_test_dict[sent] = list(cricketers_test_dict[sent])
# for sent in politicians_test_dict:
#   politicians_test_dict[sent] = list(politicians_test_dict[sent])

# for sent in matches_act:
#   matches_act[sent] = list(matches_act[sent])
# for sent in matches_cric:
#   matches_cric[sent] = list(matches_cric[sent])
for sent in matches_pol:
  matches_pol[sent] = list(matches_pol[sent])

In [ ]:
# # saving the input test files
# with open("/content//drive//MyDrive//Transformer Output//actors_test.json", "w") as outfile: 
#     json.dump(actors_test_dict,outfile)
# with open("/content/drive/MyDrive/Transformer Output/cricketers_test.json", "w") as outfile: 
#     json.dump(cricketers_test_dict, outfile)
# with open("/content/drive/MyDrive/Transformer Output/politicians_test.json", "w") as outfile: 
#     json.dump(politicians_test_dict, outfile)

# #saving the output files
# with open("/content/drive/MyDrive/Transformer Output/actors_matches.json", "w") as outfile: 
#     json.dump(matches_act, outfile)
# with open("/content/drive/MyDrive/Transformer Output/cricketers_matches.json", "w") as outfile: 
#     json.dump(matches_cric, outfile)
with open("/content/drive/MyDrive/Transformer Output/politicians_matches.json", "w") as outfile: 
    json.dump(matches_pol, outfile)

#### We do find quite a few of the matching triples to be relevant to the sentence. But, there are a few irrelevant matches as well.
Upon analysis, we think the word overlap is working better than the vector similarity approach. A possible reason can be that when we simply average out the words in a sentence, and when we average out the words in the triples and then find the similarity between these two averages, some semantic information is lost. So, triples that should have been irrelevant are also found as similar. As the word overlap method is a strictly string overlap, the relevance is much stronger.